In [2]:
from experiments.ablate_single_feature import create_diff_agg
import numpy as np

Loaded device='mps'
Loading SAEs


/Users/benlerner/work/spar-2024/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


  0%|          | 0/12 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 12/12 [00:04<00:00,  2.66it/s]


In [8]:
d_sae = 24576
num_layers = 11
num_top_feats_to_ablate_per_layer = 1

In [3]:
top_corr_flat_idx = np.load("artefacts/ablations/top_1000_pearson_per_layer_flattened_feature_idx.npy")

In [21]:
first_layer_idxes, second_layer_idxes = (
    np.array([
        np.unravel_index(top_corr_flat_idx[layer_idx], shape=(d_sae, d_sae))[ordering_idx]
        for layer_idx in range(num_layers)
    ])
    for ordering_idx in range(2)
)

In [23]:
second_layer_idxes

array([[11914, 22737, 11914, ..., 14723,  2140, 20921],
       [19595,  1151,  1151, ..., 22164, 13941, 15256],
       [22017, 10228,  6126, ...,  5161,  7935,  1626],
       ...,
       [ 4612, 22003,  4612, ..., 21342, 14041, 10852],
       [19615,  4268, 19615, ..., 14463, 12118,  7344],
       [12555,  8032, 11266, ...,  7186,  9918, 17060]])

In [24]:
full_corrs_filename = f"artefacts/similarity_measures/pearson_correlation/res_jb_sae_feature_similarity_pearson_correlation_1M_0.0_0.1.npz"
with open(full_corrs_filename, 'rb') as full_corrs_data:
    interaction_data = np.load(full_corrs_data)['arr_0']

In [27]:
layer_idx = 0
feature_pair_idx = 500
interaction_data[layer_idx, first_layer_idxes[layer_idx][feature_pair_idx], second_layer_idxes[layer_idx][feature_pair_idx]]

0.99695206

In [29]:
diff_agg = create_diff_agg(first_layer_idx=0, feature_idx=2, num_batches=2)

100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


In [31]:
import wat

In [32]:
wat / diff_agg

value: <experiments.ablate_single_feature.Aggregator object at 0x315f39220>
type: experiments.ablate_single_feature.Aggregator

Public attributes:
  m2_diffs: torch.Tensor = tensor([1.3095e-10, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 4.6038e-12,…
  masked_m2: torch.Tensor = tensor([5.3631e-07, 0.0000e+00,        nan,  ...,        nan,        nan,…
  masked_means: torch.Tensor = tensor([1.1444e-05, 0.0000e+00,        nan,  ...,        nan,        nan,…
  masked_mse: torch.Tensor = tensor([1.3097e-10, 0.0000e+00,        nan,  ...,        nan, 4.6043e-12,…
  masked_n: torch.Tensor = tensor([1., 2., 0.,  ..., 0., 1., 0.], device='mps:0')
  masked_stdevs: torch.Tensor = tensor([1.2643e-07, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 2.3706e-08,…
  masked_variances: torch.Tensor = tensor([5.3631e-07, 0.0000e+00,        nan,  ...,        nan,        nan,…
  mean_diffs: torch.Tensor = tensor([ 1.3970e-09,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,…
  min_activation_tol: float = 1e-15
  mse: 